In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [21]:
def get_tickers(file=r"Stock names.txt"):
    with open(file) as f:
        tickers = f.read().split("\n")
    return [ticker for ticker in tickers if "." not in ticker]
    return tickers

In [22]:
def get_basic_info(stock):
    # print(ticker, end="' ")
    # stock = yf.Ticker(ticker)
    data = stock.info
    div_yield = data["dividendYield"]  # * 100
    div_ex = pd.to_datetime(data["exDividendDate"], unit='s')  # Ex date
    payout = data["payoutRatio"] # * 100
    trail_div_yield = data["trailingAnnualDividendYield"]  # * 100
    avg_yield_5y = data["fiveYearAvgDividendYield"]
    
    return div_yield, trail_div_yield, div_ex, payout, avg_yield_5y

In [47]:
def get_div_info(stock):
    tmp = stock.dividends
    div_df = pd.DataFrame(index=tmp.index)
    div_df["Dividends"] = tmp
    div_df.index = div_df.index.year
    div_df = div_df.groupby(div_df.index).mean()
    grow_y = div_df.first_valid_index()
    div_df["Growth year"] = ((div_df["Dividends"]/div_df["Dividends"].shift(1)) - 1) * 100
    div_df["DGR"] = div_df["Growth year"].expanding().mean()
    div_df["DGR5y"] = div_df["Growth year"].rolling(5).mean()
    div_df["DGR3y"] = div_df["Growth year"].rolling(3).mean()
    div_df["DGR10y"] = div_df["Growth year"].rolling(10).mean()
    div_df['Years grow'] = np.where(div_df['Growth year'] >= 0, div_df['Growth year'].groupby((div_df['Growth year'] < 0).cumsum()).cumcount(), 0)
#     display(div_df)
    growth_rate = float(div_df.tail(1)["DGR"])
    growth_year = int(div_df.tail(1)["Years grow"])  # Years of growth
    growth_rate_3 = float(div_df.tail(1)["DGR3y"])
    growth_rate_5 = float(div_df.tail(1)["DGR5y"])
    growth_rate_10 = float(div_df.tail(1)["DGR10y"])
    
    return growth_rate, growth_year, growth_rate_3, growth_rate_5, growth_rate_10

In [50]:
def main():
    tickers = get_tickers(file=r"Tickers test.txt")
    # tickers = ["CCMP", "O"]
#     tickers = ["MMM"]
    
    tickers = yf.Tickers(tickers)
    df = pd.DataFrame(columns=["div yield", "Trailing annual div yield", "exDiv", "payout ratio", "5 year yield average", "DGR", "Years of grow", "DGR3y", 
                          "DGR5y", "DGR10y"])
    
    for ticker in tickers.tickers:
        # print(ticker, end=", ")
        stock = yf.Ticker(ticker)
        fail = False
        while True:
            try:
                basic_info = get_basic_info(stock)
                div_info = get_div_info(stock)
            except TypeError:
                fail = True
                break
            except ConnectionError:
                print("Connection error")
                time.sleep(15)
                continue
            break
        if fail:
            continue
        df.loc[ticker] = basic_info[0], basic_info[1], basic_info[2], basic_info[3], basic_info[4], div_info[0], div_info[1], div_info[2], div_info[3], div_info[4]
        clear_output(wait=True)
        display(df)

    df["payout ratio"] *= 100
    df["div yield"] *= 100
    df["Trailing annual div yield"] *= 100
    
    return df

In [51]:
df = main()

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,DGR,Years of grow,DGR3y,DGR5y,DGR10y
ABM,0.0192,0.016982,2023-01-04,0.2287,1.92,6.402326,39,6.051598,4.758662,3.859558
ADM,0.0215,0.019435,2023-02-08,0.2075,2.75,22.159489,24,4.581010,4.581628,8.827155
ADP,0.0226,0.019385,2023-03-09,0.5796,1.94,13.986744,39,10.109797,13.426074,11.959566
AFL,0.0230,0.020970,2023-02-14,0.1957,2.35,14.610297,38,14.257656,13.231870,9.297921
ALB,0.0056,0.005654,2022-12-15,0.1194,1.33,11.724059,23,2.400106,4.347713,7.186689
...,...,...,...,...,...,...,...,...,...,...
WPC,0.0498,0.049712,2022-12-29,1.6861,5.66,4.464977,12,0.814613,1.132269,6.285253
WST,0.0028,0.002666,2023-01-24,0.0873,0.35,7.036547,36,5.353512,5.927092,7.048439
WTRG,0.0244,0.023269,2023-02-09,0.6167,2.22,6.031261,35,7.003581,6.984215,7.553708
XOM,0.0315,0.029891,2022-11-14,0.2873,5.36,6.882998,21,1.154760,3.042357,5.063537


In [53]:
df.sort_values("div yield", ascending=False)

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,DGR,Years of grow,DGR3y,DGR5y,DGR10y
EPD,7.39,7.142857,2023-01-30,80.300003,7.52,6.200992,24,2.311594,2.428697,4.046046
ENB,6.40,8.400983,2023-02-14,126.48001,6.49,10.221437,0,6.180382,9.851278,10.176155
UVV,6.02,5.946970,2023-01-06,109.5,5.55,4.974402,37,1.078152,4.142561,4.885383
TDS,5.35,5.500000,2022-12-14,174.39,3.18,5.710178,34,2.942874,3.035886,3.906206
UHT,5.28,5.255339,2022-12-15,36.42,4.14,2.205189,35,1.449478,1.471225,1.446916
...,...,...,...,...,...,...,...,...,...,...
BMI,0.80,0.731812,2022-11-23,37.61,0.95,10.433715,18,9.929511,11.671992,9.975016
RNR,0.75,0.746951,2022-12-14,NaN,0.90,6.977038,26,2.858699,2.946280,3.201395
ROP,0.63,0.571943,2023-01-06,27.55,0.52,18.513229,26,10.010738,10.587577,15.381667
ALB,0.56,0.565396,2022-12-15,11.939999,1.33,11.724059,23,2.400106,4.347713,7.186689


In [56]:
df.sort_values("5 year yield average", ascending=False)

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,DGR,Years of grow,DGR3y,DGR5y,DGR10y
EPD,7.39,7.142857,2023-01-30,80.300003,7.52,6.200992,24,2.311594,2.428697,4.046046
ENB,6.40,8.400983,2023-02-14,126.48001,6.49,10.221437,0,6.180382,9.851278,10.176155
WPC,4.98,4.971220,2022-12-29,168.61,5.66,4.464977,12,0.814613,1.132269,6.285253
UVV,6.02,5.946970,2023-01-06,109.5,5.55,4.974402,37,1.078152,4.142561,4.885383
XOM,3.15,2.989130,2022-11-14,28.73,5.36,6.882998,21,1.154760,3.042357,5.063537
...,...,...,...,...,...,...,...,...,...,...
SPGI,0.96,0.881921,2023-02-23,27.69,0.86,11.500458,9,13.420480,15.242532,5.610804
SHW,1.05,1.066382,2022-11-17,31.09,0.83,12.108426,37,16.936400,16.676244,16.996808
BRO,0.80,0.723098,2023-02-02,17.83,0.82,10.649447,35,9.280601,8.861818,9.423141
ROP,0.63,0.571943,2023-01-06,27.55,0.52,18.513229,26,10.010738,10.587577,15.381667


In [59]:
cond = df["payout ratio"] < 70
df[cond].sort_values("div yield", ascending=False)

,div yield,Trailing annual div yield,exDiv,payout ratio,5 year yield average,DGR,Years of grow,DGR3y,DGR5y,DGR10y
UHT,5.28,5.255339,2022-12-15,36.42,4.14,2.205189,35,1.449478,1.471225,1.446916
MMM,5.17,5.248789,2022-11-17,58.55,3.38,9.025397,25,1.146427,5.013263,10.137068
LEG,4.93,4.869762,2022-12-14,64.910007,3.92,9.820423,35,3.278367,4.160447,4.329888
FLIC,4.67,4.520397,2022-12-30,30.15,3.56,7.702715,14,5.750193,7.080588,6.712093
CTBI,4.21,4.015296,2023-03-14,36.68,3.64,6.591561,34,4.333042,5.279870,4.004631
...,...,...,...,...,...,...,...,...,...,...
BRO,0.80,0.723098,2023-02-02,17.83,0.82,10.649447,35,9.280601,8.861818,9.423141
RLI,0.80,0.788607,2022-11-29,8.08,1.04,31.149708,4,74.663149,39.721821,20.695757
ROP,0.63,0.571943,2023-01-06,27.55,0.52,18.513229,26,10.010738,10.587577,15.381667
ALB,0.56,0.565396,2022-12-15,11.939999,1.33,11.724059,23,2.400106,4.347713,7.186689
